In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
)
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 数据预处理
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train = np.expand_dims(x_train, axis=-1)  # 增加通道维度
x_test = np.expand_dims(x_test, axis=-1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=50,
    width_shift_range=0.5,
    height_shift_range=0.5,
    zoom_range=0.5,
    shear_range=0.5,
    fill_mode="nearest",
)
datagen.fit(x_train)

# 回调函数
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=8, min_lr=1e-6, verbose=1
)
early_stopping = EarlyStopping(
    monitor="val_loss", patience=20, restore_best_weights=True, verbose=1
)

# 构建改进的CNN模型
# 卷积神经网络
model = Sequential(
    [
        Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1), padding="same"),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation="relu", padding="same"),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(64, (3, 3), activation="relu", padding="same"),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation="relu", padding="same"),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation="relu", padding="same"),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.4),

        Flatten(),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(10, activation="softmax"),
    ]
)

# 编译模型
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

# 训练模型
model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=200,
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // 64,
    callbacks=[reduce_lr, early_stopping],
)

# 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

# 保存模型
model.save("mnist_model_optimized_v10.keras")

/Users/xinbob/PycharmProjects/tensorflow-sample/env/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-01-09 17:29:38.853926: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-01-09 17:29:38.853958: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-01-09 17:29:38.853962: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-01-09 17:29:38.853980: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-09 17:29:38.853993: I tensorflow/core/common_runtime/pluggable_devi

Epoch 1/200


/Users/xinbob/PycharmProjects/tensorflow-sample/env/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2025-01-09 17:29:40.391006: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


937/937 ━━━━━━━━━━━━━━━━━━━━ 55s 50ms/step - accuracy: 0.1780 - loss: 2.6269 - val_accuracy: 0.8631 - val_loss: 0.5765 - learning_rate: 0.0010
Epoch 2/200
  1/937 ━━━━━━━━━━━━━━━━━━━━ 50s 54ms/step - accuracy: 0.2969 - loss: 1.7254

/Users/xinbob/PycharmProjects/tensorflow-sample/env/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


937/937 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2969 - loss: 1.7254 - val_accuracy: 0.8557 - val_loss: 0.5999 - learning_rate: 0.0010
Epoch 3/200
937/937 ━━━━━━━━━━━━━━━━━━━━ 43s 46ms/step - accuracy: 0.5130 - loss: 1.4108 - val_accuracy: 0.9346 - val_loss: 0.2069 - learning_rate: 0.0010
Epoch 4/200
937/937 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6406 - loss: 1.0887 - val_accuracy: 0.9347 - val_loss: 0.2096 - learning_rate: 0.0010
Epoch 5/200
937/937 ━━━━━━━━━━━━━━━━━━━━ 43s 46ms/step - accuracy: 0.6378 - loss: 1.0683 - val_accuracy: 0.9527 - val_loss: 0.1561 - learning_rate: 0.0010
Epoch 6/200
937/937 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6250 - loss: 1.4256 - val_accuracy: 0.9540 - val_loss: 0.1530 - learning_rate: 0.0010
Epoch 7/200
937/937 ━━━━━━━━━━━━━━━━━━━━ 45s 48ms/step - accuracy: 0.6890 - loss: 0.9315 - val_accuracy: 0.9673 - val_loss: 0.1048 - learning_rate: 0.0010
Epoch 8/200
937/937 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7500 - loss: 0.7434 -